### Analysing the inbox and sentbox of Founder of Enrorn "Jeffrey Skilling" for suspicious activities.<br>

Key points:
- Start date of the analysis is January 2000, when the stock price of Enron started to rise by mid 2000 to a record high of US$90.75 per share.<br>
- End date of the analysis is August 2001, when Jeffrey resigned as CEO of Enron. 
- Considering emails sent to less than 5 people at once for abnormalities

In [2]:
import nltk, os, glob, email, datetime
import email.utils
import time
import datetime

directory = 'data/enron/enron_mail_20150507/maildir/'

In [3]:
user_list = []

jskilling_sentmails = dict()
jskilling_inboxmails = dict()

#private_emails = dict()
name_email_map = dict()
jeffrey_folder = 'skilling-j' 

#identifying list of users
for users in os.walk(directory):
    user_list = users[1]
    break

print("Traversing through "+jeffrey_folder+" user folders...")
for user in user_list:
    if user == jeffrey_folder:
        #print(user)
        user_folder = directory + user
        private_server_mails = dict()

        #identifying sub directories under user folder
        for d in os.walk(user_folder):
            sub_dir = d[1]
            break

        # identifying folders with word sent in it
        sent_folders = []
        if sub_dir:
            for x in sub_dir:
                if x.startswith('sent'):
                    sent_folders.append(x)
                    
        # identifying folders with word inbox in it
        inbox_folders = []
        if sub_dir:
            for x in sub_dir:
                if x.startswith('inbox'):
                    inbox_folders.append(x)            

        #iterating through sent folders
        print('Iterating through sent folders')
        for sent_folder in sent_folders:
            sent_folder_files = user_folder +'/'+sent_folder+'/*'
            root, dirs, sent_folder_filenames = next(os.walk(user_folder +'/'+sent_folder))

            for filename in sent_folder_filenames:
                fname = user_folder +'/'+sent_folder+'/'+filename
                with open(fname,'r',encoding='utf-8',errors='ignore') as file:
                    #whole email
                    message = email.message_from_string(file.read())

                    #keys
                    #print(message.keys())
                    
                    #to
                    if 'To' in message.keys():
                        to_emails = message.get('To')
                        to_email_list = to_emails.split(',')
                        
                        if(len(to_email_list) < 5):
                                                        
                            #date
                            #initial analysis shows date in PDT/PST
                            sdate = message.get('Date')
                            # Fri, 16 Feb 2001 09:26:00 -0800 (PST)
                            #datetime_object = datetime.datetime.strptime(sdate, '%b %d %Y %I:%M%p')
                            #print(sdate.datetime())
                            pdate = email.utils.parsedate(sdate)
                            make_time = time.mktime(pdate)
                            parse_date = datetime.datetime.fromtimestamp(make_time)
                            #print(parse_date) 1999-02-16 06:39:00
                            
                            #condition
                            if(parse_date > datetime.datetime(2000,1,1) and parse_date < datetime.datetime(2001,8,14)):
                            
                                #Checking for month entry in dictionary
                                date_entry = lambda parse_date: parse_date.strftime("%B")+'-'+parse_date.strftime("%Y")
                                month_entry = dict()
                                if date_entry in jskilling_sentmails.keys():
                                    month_entry = jskilling_sentmails[date_entry]

                                #from
                                if 'From' in message.keys():
                                    from_email = message.get('From')
                                if 'X-From' in message.keys():
                                    from_name = message.get('X-From')

                                #body
                                if not message.is_multipart():
                                    body = message.get_payload()

                                #storing sender email in map
                                if from_email not in name_email_map:
                                    name_email_map[from_email] = from_name

                                #filtering to_email_list
                                for to_email in to_email_list:
                                    to_email = to_email.replace('\n','').replace('\t','').strip()
                                    #if not to_email.endswith('@enron.com'):
                                    if to_email in month_entry.keys():
                                        month_entry[to_email] = month_entry[to_email] + 1
                                    else:
                                        month_entry[to_email] = 1
                                    #if to_email in private_server_mails:
                                    #    private_server_mails[to_email] = private_server_mails[to_email] + 1
                                    #else:
                                    #    private_server_mails[to_email] = 1

                                jskilling_sentmails[date_entry] = month_entry
                                #print(from_email)
                                #print(to_email)
                                #print(message.get_payload())
                                #print(to_emails)
                                #print(from_email)
                                #print(from_name) 

        #end of sent folder iteration            
        #private_emails[from_email] = private_server_mails
        
        
        #iterating through inbox folders 
        print('Iterating through inbox folders')
        for inbox_folder in inbox_folders:
            inbox_folder_files = user_folder +'/'+inbox_folder+'/*'
            root, dirs, inbox_folder_filenames = next(os.walk(user_folder +'/'+inbox_folder))

            for filename in inbox_folder_filenames:
                fname = user_folder +'/'+inbox_folder+'/'+filename
                with open(fname,'r',encoding='utf-8',errors='ignore') as file:
                    #whole email
                    message = email.message_from_string(file.read())

                    #keys
                    #print(message.keys())
                    
                    #to
                    to_email = message.get('To')

                    #date
                    #initial analysis shows date in PDT/PST
                    sdate = message.get('Date')
                    # Fri, 16 Feb 2001 09:26:00 -0800 (PST)
                    #datetime_object = datetime.datetime.strptime(sdate, '%b %d %Y %I:%M%p')
                    #print(sdate.datetime())
                    pdate = email.utils.parsedate(sdate)
                    make_time = time.mktime(pdate)
                    parse_date = datetime.datetime.fromtimestamp(make_time)
                    #print(parse_date) 1999-02-16 06:39:00

                    #condition PENDING
                    if(parse_date > datetime.datetime(2000,1,1) and parse_date < datetime.datetime(2001,8,14)):

                        #Checking for month entry in dictionary
                        date_entry = parse_date.strftime("%B")+'-'+parse_date.strftime("%Y")
                        month_entry = dict()
                        if date_entry in jskilling_inboxmails.keys():
                            month_entry = jskilling_inboxmails[date_entry]

                        #from
                        if 'From' in message.keys():
                            from_email = message.get('From')
                        if 'X-From' in message.keys():
                            from_name = message.get('X-From')

                        #body
                        if not message.is_multipart():
                            body = message.get_payload()

                        #storing sender email in map
                        #if from_email not in name_email_map:
                        #    name_email_map[from_email] = from_name

                        #filtering to_email_list
                        from_email = from_email.replace('\n','').replace('\t','').strip()
                        #if not to_email.endswith('@enron.com'):
                        if from_email in month_entry.keys():
                            month_entry[from_email] = month_entry[from_email] + 1
                        else:
                            month_entry[from_email] = 1
                        #if to_email in private_server_mails:
                        #    private_server_mails[to_email] = private_server_mails[to_email] + 1
                        #else:
                        #    private_server_mails[to_email] = 1

                        jskilling_inboxmails[date_entry] = month_entry
                        #print(from_email)
                        #print(to_email)
                        #print(message.get_payload())
                        #print(to_emails)
                        #print(from_email)
                        #print(from_name) 

        #end of sent folder iteration            
        #private_emails[from_email] = private_server_mails
        
    

Traversing through skilling-j user folders...


In [4]:
#print(jskilling_sentmails)
print('\nJeffrey Skillings frequently contacted emails after stock hike and before resignation.')
print('Top 5 people contacted in each month:')
most_contacted_count = 0
most_contacted_email = ''
total_emails = 0
for month,entries in jskilling_sentmails.items():
    print('')
    print(month)
    mail_count = 0
    sorted_entries = [(k, entries[k]) for k in sorted(entries, key=entries.get, reverse=True)]
    c = 0
    for entry in sorted_entries:
        if c < 5:
            print(str(entry[1])+' mailto '+ entry[0])
            c += 1
        mail_count = mail_count + count
        if(most_contacted_count < entry[1]):
            most_contacted_count = entry[1]
            most_contacted_email = entry[0]
    print('('+str(mail_count)+' mail)')   
    total_emails = total_emails + mail_count

print('\nConclusion')
print('Total sent emails :' +str(total_emails))
print(str(most_contacted_count)+' mailsto '+most_contacted_email)
print('Insight Partners is a private equity and venture capital firm based in New York.')


Jeffrey Skillings frequently contacted emails after stock hike and before resignation.
Top 5 people contacted in each month:

Conclusion
Total sent emails :0
0 mailsto 
Insight Partners is a private equity and venture capital firm based in New York.


In [55]:
#print(jskilling_sentmails)
print('\nPeople who have contacted Jeffrey Skillings after stock hike and before his resignation')
print('Top 5 people from each month:')

most_contacted_count = 0
most_contacted_email = ''
total_emails = 0

for month,entries in jskilling_inboxmails.items():
    print('')
    print(month)
    mail_count = 0
    sorted_entries = [(k, entries[k]) for k in sorted(entries, key=entries.get, reverse=True)]
    c = 0
    for entry in sorted_entries:
        if c < 5:
            print(str(entry[1])+' mailto '+ entry[0])
            c += 1
        mail_count = mail_count + count
        if(most_contacted_count < entry[1]):
            most_contacted_count = entry[1]
            most_contacted_email = entry[0]
    print('('+str(mail_count)+' mail)')  
    total_emails = total_emails + mail_count

print('\nConclusion')    
print('Total recieved emails :' +str(total_emails))
print(str(most_contacted_count)+' mailsby '+most_contacted_email)


People who have contacted Jeffrey Skillings after stock hike and before his resignation
Top 5 people from each month:

June-2001
9 mailto no.address@enron.com
6 mailto david.morris@lehman.com
4 mailto la.rose@enron.com
3 mailto christie.patrick@enron.com
3 mailto svarga@kudlow.com
(87 mail)

April-2001
15 mailto amelia.alder@enron.com
4 mailto outlook.team@enron.com
3 mailto david.delainey@enron.com
3 mailto stephanie.strong@enron.com
3 mailto lou.moore@enron.com
(94 mail)

May-2001
10 mailto no.address@enron.com
5 mailto kevinscott@onlinemailbox.net
4 mailto billy.lemmons@enron.com
4 mailto erica.harris@enron.com
4 mailto perfmgmt@enron.com
(157 mail)

August-2001
2 mailto boethia@hotmail.com
2 mailto christie.patrick@enron.com
1 mailto bprzyby@evokesoft.com
1 mailto david.pruner@enron.com
1 mailto jdent@mmcable.com
(23 mail)

July-2001
3 mailto boethia@hotmail.com
2 mailto tskilling@tribune.com
2 mailto dmusslewhite@sterlingmccalltoyota.com
2 mailto dal.communityaffairs@dal.frb.org


In [57]:
print("\nJeffrey Skilling Resigned in  August 2001.")


Jeffrey Skilling Resigned in  August 2001.
